<a href="https://colab.research.google.com/github/Imamahdysabilla/Machine-Learning/blob/main/mobil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files

# Unggah file
uploaded = files.upload()


Saving parkir.jpeg to parkir (1).jpeg


In [6]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

# Periksa apakah file sudah diunggah
if not os.path.exists("yolov3.weights") or not os.path.exists("yolov3.cfg") or not os.path.exists("coco.names"):
    print("Pastikan file yolov3.weights, yolov3.cfg, dan coco.names sudah diunggah")
else:
    # Load model YOLO
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # Load COCO class labels
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Fungsi untuk mendeteksi mobil dalam gambar
    def detect_cars(image_path):
        # Load gambar
        image = cv2.imread(image_path)
        height, width, channels = image.shape

        # Persiapkan gambar untuk model
        blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Analisis hasil keluaran
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and classes[class_id] == "car":
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        car_count = len(indexes)
        print(f"Terdeteksi {car_count} mobil.")

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = (0, 255, 0)
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # Tampilkan gambar hasil
        cv2_imshow(image)

    # Contoh penggunaan
    detect_cars("parkir.jpeg")


Pastikan file yolov3.weights, yolov3.cfg, dan coco.names sudah diunggah
